In [ ]:
# pip install undetected-chromedriver selenium
import os, time, random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.common.exceptions import TimeoutException

EMAIL = os.getenv("LBC_EMAIL", "email@gmail.com")
PASSWORD = os.getenv("LBC_PASSWORD", "motDePasse")

LOGIN_URL  = "https://auth.leboncoin.fr/login"
SEARCH_URL = ("https://www.leboncoin.fr/recherche?"
              "category=9&locations=Saint-Etienne__45.424942780483555_4.4207816810634_7579_5000"
              "&price=min-75000&real_estate_type=2&owner_type=all&sort=price&order=asc&kst=r&from=rs")

def pause(a=1.35, b=1.9): time.sleep(random.uniform(a, b))

def click_hard(driver, el):
    try:
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
        ActionChains(driver).move_to_element(el).pause(0.15).click().perform()
        return True
    except Exception:
        try:
            driver.execute_script("""
              const el = arguments[0];
              ['pointerdown','mousedown','mouseup','click'].forEach(t=>{
                el.dispatchEvent(new MouseEvent(t,{bubbles:true,cancelable:true,view:window}));
              });
            """, el)
            return True
        except Exception:
            return False

opts = uc.ChromeOptions()
opts.add_argument("--start-maximized")
driver = uc.Chrome(options=opts)
wait = WebDriverWait(driver, 20)

try:
    driver.get(LOGIN_URL)
    pause()

    # 1) EMAIL + "Continuer"
    email_inp = wait.until(EC.visibility_of_element_located((By.NAME, "email")))
    email_inp.clear(); email_inp.send_keys(EMAIL)
    pause()

    # bouton "Continuer" (plusieurs variantes)
    cont_btn = wait.until(EC.element_to_be_clickable((
        By.XPATH,
        "//button[contains(.,'Continuer')] | //button[@type='submit' and not(contains(.,'Se connecter'))]"
    )))
    if not click_hard(driver, cont_btn): cont_btn.send_keys(Keys.RETURN)
    pause()

    # 2) MOT DE PASSE + "Se connecter"
    pwd_inp = wait.until(EC.visibility_of_element_located((By.NAME, "password")))
    pwd_inp.clear(); pwd_inp.send_keys(PASSWORD)
    pause()

    login_btn = wait.until(EC.element_to_be_clickable((
        By.XPATH,
        "//button[contains(.,'Se connecter')] | //button[@type='submit' and contains(.,'connecter')]"
    )))
    if not click_hard(driver, login_btn): pwd_inp.send_keys(Keys.RETURN)
    pause(0.8, 1.4)

    # 3) Aller sur la recherche (tu peux supprimer si tu veux juste tester le login)
    driver.get(SEARCH_URL)
    pause(1.2, 1.8)

    print("✅ Connexion OK et navigation vers la recherche.")

except TimeoutException as e:
    driver.save_screenshot("lbc_timeout.png")
    print("⛔ Timeout sur un élément (capture: lbc_timeout.png) :", e)
finally:
    # driver.quit()
    pass